<a href="https://colab.research.google.com/github/allaboutsimple/aibootcamp/blob/main/n222a_random_forests_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 2 / SPRINT 2 / NOTE 2*

# 📝 Assignment
---

In [9]:
'''pip install kaggle --upgrade
import os

os.environ['KAGGLE_USERNAME'] = 'simplecode'
os.environ['KAGGLE_KEY'] = 'd51973b533c839f65e363dc8ea47b938'

!kaggle -h
!kaggle competitions download -c prediction-of-h1n1-vaccination
!unzip '*.zip'
!ls'''

"pip install kaggle --upgrade\nimport os\n\nos.environ['KAGGLE_USERNAME'] = 'simplecode'\nos.environ['KAGGLE_KEY'] = 'd51973b533c839f65e363dc8ea47b938'\n\n!kaggle -h\n!kaggle competitions download -c prediction-of-h1n1-vaccination\n!unzip '*.zip'\n!ls"

# 랜덤포레스트(Random Forests)

### 1) 캐글 대회를 이어서 진행합니다. EDA, 데이터 전처리 부분을 업데이트 하세요.
- EDA는 항상 완벽할 수 없지요 한 주간 계속 같은 데이터로 과제를 진행하므로 부족한 부분을 추가하거나 논하세요.
- (지금은 feature engineering에 너무 시간을 들이지 마세요!)
- Ordinal Encoding을 적용해 보세요.
- **(객관식) 다음 특성들 중에 순서를 고려한 Ordinal Encoding이 필요해 보이는 특성을 선택하고 과제 제출폼에 제출하세요.**
    1. opinion_h1n1_vacc_effective
    2. state
    3. marital
    4. employment_occupation


In [10]:
import pandas as pd
import numpy as np

In [11]:
target = 'vacc_h1n1_f'
train = pd.merge(pd.read_csv('/train.csv'),
                 pd.read_csv('/train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('/test.csv')
sample_submission = pd.read_csv('/submission.csv')

In [12]:
train.dtypes

h1n1_concern                   float64
h1n1_knowledge                 float64
behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
doctor_recc_h1n1               float64
doctor_recc_seasonal           float64
chronic_med_condition          float64
child_under_6_months           float64
health_insurance               float64
health_worker                  float64
opinion_h1n1_vacc_effective     object
opinion_h1n1_risk               object
opinion_h1n1_sick_from_vacc     object
opinion_seas_vacc_effective     object
opinion_seas_risk               object
opinion_seas_sick_from_vacc     object
agegrp                          object
education_comp                 float64
raceeth4_i                       int64
sex_i                            int64
inc_pov                  

In [13]:
check = train.T

In [14]:
check.loc[['opinion_h1n1_vacc_effective','marital','employment_occupation','state']]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,42114,42115,42116,42117,42118,42119,42120,42121,42122,42123,42124,42125,42126,42127,42128,42129,42130,42131,42132,42133,42134,42135,42136,42137,42138,42139,42140,42141,42142,42143,42144,42145,42146,42147,42148,42149,42150,42151,42152,42153
opinion_h1n1_vacc_effective,NaN,Very Effective,Somewhat Effective,Very Effective,Very Effective,Very Effective,NaN,Not Very Effective,Dont Know,NaN,Not Very Effective,NaN,Somewhat Effective,Very Effective,Somewhat Effective,Dont Know,Dont Know,Somewhat Effective,Somewhat Effective,Not Very Effective,Somewhat Effective,Somewhat Effective,NaN,Dont Know,NaN,NaN,Not At All Effective,NaN,NaN,Somewhat Effective,Somewhat Effective,Not Very Effective,Very Effective,Very Effective,Somewhat Effective,Very Effective,Very Effective,Dont Know,Somewhat Effective,Somewhat Effective,...,Somewhat Effective,Somewhat Effective,Somewhat Effective,Very Effective,Somewhat Effective,Very Effective,Dont Know,Dont Know,Very Effective,Not Very Effective,Somewhat Effective,Very Effective,Somewhat Effective,Not At All Effective,Somewhat Effective,NaN,Somewhat Effective,Not At All Effective,NaN,Somewhat Effective,Somewhat Effective,NaN,NaN,Somewhat Effective,Somewhat Effective,NaN,Somewhat Effective,Very Effective,Somewhat Effective,Somewhat Effective,Somewhat Effective,Very Effective,Dont Know,Dont Know,NaN,Somewhat Effective,Very Effective,Somewhat Effective,Somewhat Effective,NaN
marital,NaN,2,2,1,1,1,NaN,2,1,NaN,1,NaN,1,2,2,1,1,NaN,1,2,2,1,NaN,1,NaN,NaN,1,NaN,NaN,1,1,2,2,2,1,1,1,2,1,1,...,1,1,2,1,2,2,2,NaN,2,NaN,1,1,1,2,1,NaN,1,1,NaN,2,2,NaN,NaN,2,1,NaN,2,1,2,2,1,1,1,2,NaN,1,1,2,1,NaN
employment_occupation,NaN,Food Preparation and Serving Related Occupations,Transportation and Material Moving Occupations,Legal Occupations,Office and Administrative Support Occupations,Not ascertained,NaN,Management Occupations,Transportation and Material Moving Occupations,NaN,NaN,NaN,Architecture and Engineering Occupations,Business and Financial Operations Occupations,Building and Grounds Cleaning and Maintenance ...,Sales and Related Occupations,NaN,NaN,"Life, Physical, and Social Science Occupations",NaN,"Education, Training, and Library Occupations",NaN,NaN,"Arts, Design, Entertainment, Sports and Media ...",NaN,NaN,NaN,NaN,NaN,Food Preparation and Serving Related Occupations,"Arts, Design, Entertainment, Sports and Media ...",Food Preparation and Serving Related Occupations,NaN,Management Occupations,Business and Financial Operations Occupations,NaN,NaN,NaN,NaN,Office and Administrative Support Occupations,...,Architecture and Engineering Occupations,NaN,Sales and Related Occupations,NaN,Sales and Related Occupations,Healthcare Practitioners and Technical Occupat...,NaN,NaN,Sales and Related Occupations,NaN,NaN,Not ascertained,NaN,Healthcare Support Occupations,NaN,NaN,Not ascertained,NaN,NaN,NaN,Management Occupations,NaN,NaN,Office and Administrative Support Occupations,Management Occupations,NaN,NaN,Transportation and Material Moving Occupations,Healthcare Support Occupations,Business and Financial Operations Occupations,NaN,"Education, Training, and Library Occupations",Sales and Related Occupations,Business and Financial Operations Occupations,NaN,NaN,Healthcare Practitioners and Technical Occupat...,Office and Administrative Support Occupations,Healthcare Practitioners and Technical Occupat...,NaN
state,CALIFORNIA,NEBRASKA,MASSACHUSETTS,LOUISIANA,ALABAMA,MONTANA,MISSOURI,MISSOURI,WISCONSIN,FLORIDA,MISSOURI,ALABAMA,MASSACHUSETTS,MARYLAND,NEVADA,LOUISIANA,PENNSYLVANIA,FLORIDA,RHODE ISLAND,ARIZONA,MAINE,SOUTH DAKOTA,WYOMING,CONNECTICUT,NEVADA,RHODE ISLAND,DELAWARE,SOUTH DAKOTA,NEW YORK,SOUTH CAROLINA,ALABAMA,NEW JERSEY,NORTH CAROLINA,DISTRICTOF COLUMBIA,NEW HAMPSHIRE,SOUTH CAROLINA,TEXAS,MAINE,OREGON,KENTUCKY,...,NEBRASKA,WASHINGTON,NEVADA,MICHIGAN,MARYLAND,VIRGINIA,OREGON,RHODE ISLAND,MARYLAND,DISTRICTOF COLUMB

In [15]:
train['opinion_h1n1_vacc_effective'].unique()

array([nan, 'Very Effective', 'Somewhat Effective', 'Not Very Effective',
       'Dont Know', 'Not At All Effective', 'Refused'], dtype=object)

In [16]:
! pip install --upgrade category_encoders

     |████████████████████████████████| 81kB 3.5MB/s 


In [17]:
from category_encoders import OrdinalEncoder
enc = OrdinalEncoder(handle_missing="value")
X = [['Very Effective', 6,'Yes'], ['Somewhat Effective', 5,'Yes'], ['Not Very Effective',4,'No'], ['Dont Know',3,'None'],
      ['Not At All Effective',2,'No'], ['Refused',1,'No']]
enc.fit(X)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


OrdinalEncoder(cols=[0, 2], drop_invariant=False, handle_missing='value',
               handle_unknown='value',
               mapping=[{'col': 0, 'data_type': dtype('O'),
                         'mapping': Very Effective          1
Somewhat Effective      2
Not Very Effective      3
Dont Know               4
Not At All Effective    5
Refused                 6
NaN                    -2
dtype: int64},
                        {'col': 2, 'data_type': dtype('O'),
                         'mapping': Yes     1
No      2
None    3
NaN    -2
dtype: int64}],
               return_df=True, verbose=0)

In [18]:
enc.transform( [['Very Effective', 6,'Yes'], ['Somewhat Effective', 5,'Yes'], ['Not Very Effective',4,'No'], ['Dont Know',3,'None'],
      ['Not At All Effective',2,'No'], ['Refused',1,'No']])

,0,1,2
0,1,6,1
1,2,5,1
2,3,4,2
3,4,3,3
4,5,2,2
5,6,1,2


In [19]:
enc.category_mapping

[{'col': 0, 'data_type': dtype('O'), 'mapping': Very Effective          1
  Somewhat Effective      2
  Not Very Effective      3
  Dont Know               4
  Not At All Effective    5
  Refused                 6
  NaN                    -2
  dtype: int64}, {'col': 2, 'data_type': dtype('O'), 'mapping': Yes     1
  No      2
  None    3
  NaN    -2
  dtype: int64}]

In [20]:
test.dtypes

h1n1_concern                   float64
h1n1_knowledge                 float64
behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
doctor_recc_h1n1               float64
doctor_recc_seasonal           float64
chronic_med_condition          float64
child_under_6_months           float64
health_insurance               float64
health_worker                  float64
opinion_h1n1_vacc_effective     object
opinion_h1n1_risk               object
opinion_h1n1_sick_from_vacc     object
opinion_seas_vacc_effective     object
opinion_seas_risk               object
opinion_seas_sick_from_vacc     object
agegrp                          object
education_comp                 float64
raceeth4_i                       int64
sex_i                            int64
inc_pov                  

In [21]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train[target], random_state=2)


train.shape, val.shape, test.shape

((33723, 39), (8431, 39), (28104, 38))

In [22]:
def engineer(df):
    """특성을 엔지니어링 하는 함수입니다."""
    
    # 높은 카디널리티를 가지는 특성을 제거합니다.
    selected_cols = df.select_dtypes(include=['number', 'object'])
    colnames = selected_cols.columns.tolist()
    labels = selected_cols.nunique()
    
    selected_features = labels[labels <= 30].index.tolist()
    df = df[selected_features]
    
    # 새로운 특성을 생성합니다.
    behaviorals = [col for col in df.columns if 'behavioral' in col] 
    df['behaviorals'] = df[behaviorals].sum(axis=1)
    
    
    dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
    df.drop(columns=dels, inplace=True)
        
    return df


train = engineer(train)
val = engineer(val)
test = engineer(test)

In [23]:
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [24]:

%%time
from category_encoders import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

# ordinal encoding
pipe_ord = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=10, n_jobs=-1, oob_score=True)
)

pipe_ord.fit(X_train, y_train)
print('검증 정확도', pipe_ord.score(X_val, y_val))

검증 정확도 0.8239829201755426
CPU times: user 6.73 s, sys: 169 ms, total: 6.9 s
Wall time: 4.2 s


In [25]:
practice = X_train.head().T
practice

,20365,29138,36128,39793,32973
h1n1_concern,1,1,2,2,0
h1n1_knowledge,1,1,2,1,1
behavioral_antiviral_meds,0,0,0,0,0
behavioral_avoidance,0,1,1,1,0
behavioral_face_mask,0,0,0,0,0
behavioral_wash_hands,0,1,1,1,0
behavioral_large_gatherings,0,1,1,0,0
behavioral_outside_home,0,1,1,0,0
behavioral_touch_face,0,1,0,1,0
doctor_recc_h1n1,0,1,0,0,0


### 2) 랜덤포레스트 모델을 적용한 후의 결과를 캐글에 제출하세요.
- 랜덤포레스트를 적용하고 성능이 오히려 떨어졌을 수도 있습니다! 만약 그렇다면 이유를 본인 논리로 분석해 보세요.
- **캐글 Leaderboard의 본인 Score를 과제 제출폼에 제출하세요.**

In [26]:
%%time
from category_encoders import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    OneHotEncoder(use_cat_names=True), 
    SimpleImputer(), 
    RandomForestClassifier(n_jobs=-1, random_state=10, oob_score=True)
)

pipe.fit(X_train, y_train)
print('검증 정확도: ', pipe.score(X_val, y_val))

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


검증 정확도:  0.8270667773692326
CPU times: user 7.39 s, sys: 162 ms, total: 7.55 s
Wall time: 4.61 s


In [27]:
pipe.named_steps['randomforestclassifier'].oob_score_

0.8164754025442577

In [28]:
#학습시키기
model = pipe_ord.fit(X_train,y_train)

y_pred = model.predict(X_test)
target = 'vacc_h1n1_f'
submission = sample_submission
submission[target] = y_pred

#Export
submission.to_csv("./first_submission.csv", index=False, header=1)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


## 🔥 도전과제

### 3) 수업에 사용하지 않은 다른 종류의 [category_encoders](http://contrib.scikit-learn.org/category_encoders/)을 적용해 보고 인코더에 대해 설명하세요


In [29]:
### 이곳에서 과제를 진행해 주세요 ###

## 참고자료

- [Random Forests for Complete Beginners](https://victorzhou.com/blog/intro-to-random-forests/)